In [31]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from scipy.optimize import curve_fit

from src.constants import (
	DIR_DATA,
	DIR_GRAPH,
	DIR_PICKLE,
	FILE_DATA_FRAME,
	SOLUTION_DP,
  SOLUTION_GREEDY,
	SOLUTION_CHRISTOFIDES,
	SOLUTION_SIMULATED_ANNEALING_RANDOM_SWAP,
	SOLUTION_SIMULATED_ANNEALING_NEAREST_NEIGHBOR_2_OPT,
  SOLUTION_SIMULATED_ANNEALING_GREEDY_SWAP,
  SOLUTION_SIMULATED_ANNEALING_LOGARITHMIC_COOLING_2_OPT,
  SOLUTION_SIMULATED_ANNEALING_GREEDY_3_OPT,
	SOLUTION_LIN_KERNIGHAN,
	SOLUTION_GENETIC_TOURNAMENT_SELECTION,
	SOLUTION_GENETIC_ADAPTIVE_MUTATION,
	ALL_SOLUTIONS
)

from src.utils import (
	print_time_taken,
	process_all_files_to_png,
	process_all_files_to_dataframe,
	save_dataframe_to_file,
	load_dataframe_from_file
)

from src.solver.dynamic_programming_tsp_solver import DynamicProgrammingTspSolver
from src.solver.greedy_tsp_solver import GreedyTspSolver
from src.solver.christofides_tsp_solver import ChristofidesTspSolver
from src.solver.sa.random_swap_simulated_annealing_tsp_solver import RandomSwapSimulatedAnnealingSolver
from src.solver.sa.nearest_neighbor_two_opt_simulated_annealing_tsp_solver import NearestNeighborTwoOptSimulatedAnnealingSolver
from src.solver.sa.greedy_swap_simulated_annealing_tsp_solver import GreedySwapSimulatedAnnealingSolver
from src.solver.sa.logarithmic_cooling_two_opt_simulated_annealing_tsp_solver import LogarithmicCoolingTwoOptSimulatedAnnealingSolver
from src.solver.sa.greedy_three_opt_simulated_annealing_tsp_solver import GreedyThreeOptSimulatedAnnealingSolver
from src.solver.ga.tournament_selection_genetic_tsp_solver import TournamentSelectionGeneticAlgorithmSolver
from src.solver.ga.adaptive_mutation_genetic_tsp_solver import AdaptiveMutationGeneticAlgorithmSolver
from src.solver.lin_kernighan_tsp_solver import LinKernighanSolver

# Pre-Processing

## Creating Graphs for Visualization

In [2]:
# Create the output "graph" directory if it doesn't exist
if not os.path.exists(DIR_GRAPH):
	os.makedirs(DIR_GRAPH)

# Process all XML files in "data" directory and generate graphs
# process_all_files_to_png(DIR_DATA, DIR_GRAPH)

## Constructing DataFrame of Problem-Graphs

In [3]:
# Process all the files in the "data" directory into a DataFrame
# df = process_all_files_to_dataframe(DIR_DATA)

# Create the output "pickle" directory if it doesn't exist
if not os.path.exists(DIR_PICKLE):
	os.makedirs(DIR_PICKLE)

# Save the DataFrame to a "tsp_problems_dataframe.pkl" file
# save_dataframe_to_file(df, FILE_DATA_FRAME)

# Load the DataFrame from the file
df_loaded = load_dataframe_from_file(FILE_DATA_FRAME)

# Display the first few rows to confirm
df_loaded.head()

DataFrame loaded successfully.


,tsp_problem,number_of_vertices,graph,optimal_cost
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0


# TSP Solutions

In [4]:
def solve_tsp(df, solver, algorithm_name):
	# Check if the new columns exists, if not, create them
	create_performance_metric_columns_if_not_exist(df, algorithm_name)

	num_of_problems = len(df.index)
	# Loop through each row in the dataframe and solve the TSP problem using TspSolver
	for index, row in df.iterrows():
		graph = row['graph']
		problem_name = row['tsp_problem']
		
		print(f"[{index + 1}/{num_of_problems}] Solving TSP using \"{algorithm_name}\" for problem: {problem_name} with {row['number_of_vertices']} vertices...")
		
		# Solve the TSP using the solver
		start = time.time()
		solution = solver.solve(graph)
		end = time.time()

		if solution:
			print_time_taken(f"solve TSP using \"{algorithm_name}\" for problem: {problem_name}", start, end)

			# Obtain/calculate performance metrics
			tour, total_cost = solution
			execution_time = end - start
			optimal_cost = row.get('optimal_cost', None)
			deviation_from_optimal = calculate_deviation_from_optimal_cost(total_cost, optimal_cost)

		else:
			print("Did not solve.")

			# Set performance metrics to n/a values
			tour = None
			total_cost = np.nan
			execution_time = np.nan
			optimal_cost = np.nan
			deviation_from_optimal = np.nan

		print("=====================================")

		# Update the dataframe
		update_data_frame_row_with_performance_metrics(df, index, algorithm_name, tour, total_cost, execution_time, deviation_from_optimal)

In [5]:
def create_performance_metric_columns_if_not_exist(df, algorithm_name):
	if f'{algorithm_name}_tour' not in df.columns:
		df[f'{algorithm_name}_tour'] = pd.Series(dtype='object')
	if f'{algorithm_name}_cost' not in df.columns:
		df[f'{algorithm_name}_cost'] = pd.Series(dtype='float')
	if f'{algorithm_name}_execution_time' not in df.columns:
		df[f'{algorithm_name}_execution_time'] = pd.Series(dtype='float')
	if f'{algorithm_name}_deviation_from_optimal' not in df.columns:
		df[f'{algorithm_name}_deviation_from_optimal'] = pd.Series(dtype='float')

In [6]:
def calculate_deviation_from_optimal_cost(total_cost, optimal_cost):
	if optimal_cost is not None:
		return (total_cost - optimal_cost) / optimal_cost * 100

In [7]:
def update_data_frame_row_with_performance_metrics(df, index, algorithm_name, tour, total_cost, execution_time, deviation_from_optimal):
	df.at[index, f'{algorithm_name}_tour'] = tour
	df.at[index, f'{algorithm_name}_cost'] = total_cost
	df.at[index, f'{algorithm_name}_execution_time'] = execution_time
	df.at[index, f'{algorithm_name}_deviation_from_optimal'] = deviation_from_optimal

## Exact Algorithms

### Held-Karp Algorithm (Dynamic Programming)

In [8]:
solve_tsp(df_loaded, DynamicProgrammingTspSolver(), SOLUTION_DP)

df_loaded.head()

[1/72] Solving TSP using "dp" for problem: ch130 with 130 vertices...
Did not solve.
[2/72] Solving TSP using "dp" for problem: gr666 with 666 vertices...
Did not solve.
[3/72] Solving TSP using "dp" for problem: kroB100 with 100 vertices...
Did not solve.
[4/72] Solving TSP using "dp" for problem: d198 with 198 vertices...
Did not solve.
[5/72] Solving TSP using "dp" for problem: kroA150 with 150 vertices...
Did not solve.
[6/72] Solving TSP using "dp" for problem: tsp225 with 225 vertices...
Did not solve.
[7/72] Solving TSP using "dp" for problem: fl417 with 417 vertices...
Did not solve.
[8/72] Solving TSP using "dp" for problem: brg180 with 180 vertices...
Did not solve.
[9/72] Solving TSP using "dp" for problem: brazil58 with 58 vertices...
Did not solve.
[10/72] Solving TSP using "dp" for problem: lin105 with 105 vertices...
Did not solve.
[11/72] Solving TSP using "dp" for problem: gil262 with 262 vertices...
Did not solve.
[12/72] Solving TSP using "dp" for problem: pcb442 wit

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN


## Approximation and Heuristic Algorithms

### Greedy TSP

In [9]:
solve_tsp(df_loaded, GreedyTspSolver(), SOLUTION_GREEDY)

df_loaded.head()

[1/72] Solving TSP using "greedy" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "greedy" for problem: ch130: 0.03126s
[2/72] Solving TSP using "greedy" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "greedy" for problem: gr666: 0.57468s
[3/72] Solving TSP using "greedy" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "greedy" for problem: kroB100: 0.00309s
[4/72] Solving TSP using "greedy" for problem: d198 with 198 vertices...
Time taken to solve TSP using "greedy" for problem: d198: 0.01801s
[5/72] Solving TSP using "greedy" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "greedy" for problem: kroA150: 0.00883s
[6/72] Solving TSP using "greedy" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "greedy" for problem: tsp225: 0.02579s
[7/72] Solving TSP using "greedy" for problem: fl417 with 417 vertices...
Time taken to solve TSP using "greedy" for problem: fl417: 0.14603

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,0.031263,23.981772
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,0.574678,24.665204
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,0.003091,31.678983
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,0.018013,17.997933
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,0.008833,26.714926


## Christofides Algorithm

In [10]:
solve_tsp(df_loaded, ChristofidesTspSolver(), SOLUTION_CHRISTOFIDES)

df_loaded.head()

[1/72] Solving TSP using "christofides" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "christofides" for problem: ch130: 0.20312s
[2/72] Solving TSP using "christofides" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "christofides" for problem: gr666: 22.96708s
[3/72] Solving TSP using "christofides" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "christofides" for problem: kroB100: 0.04679s
[4/72] Solving TSP using "christofides" for problem: d198 with 198 vertices...
Time taken to solve TSP using "christofides" for problem: d198: 0.84444s
[5/72] Solving TSP using "christofides" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "christofides" for problem: kroA150: 0.34077s
[6/72] Solving TSP using "christofides" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "christofides" for problem: tsp225: 0.55476s
[7/72] Solving TSP using "christofides" for problem: fl417 with 417

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal,christofides_tour,christofides_cost,christofides_execution_time,christofides_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,0.031263,23.981772,"[0, 48, 52, 119, 59, 50, 41, 43, 39, 46, 36, 9...",1.784470e+04,0.203121,192.057244
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,0.574678,24.665204,"[0, 665, 107, 108, 109, 110, 111, 449, 461, 46...",1.750465e+06,22.967080,494.672134
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,0.003091,31.678983,"[0, 89, 20, 16, 77, 12, 37, 19, 79, 29, 50, 47...",5.950318e+04,0.046793,168.746557
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,0.018013,17.997933,"[0, 119, 123, 124, 168, 61, 52, 136, 143, 144,...",7.218672e+04,0.844442,357.457031
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,0.008833,26.714926,"[0, 70, 40, 65, 64, 123, 117, 127, 42, 96, 142...",1.203042e+05,0.340766,353.567357


## Simulated Annealing

### Random Swap

In [11]:
solve_tsp(df_loaded, RandomSwapSimulatedAnnealingSolver(), SOLUTION_SIMULATED_ANNEALING_RANDOM_SWAP)

df_loaded.head()

[1/72] Solving TSP using "sa-random_swap" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "sa-random_swap" for problem: ch130: 0.05408s
[2/72] Solving TSP using "sa-random_swap" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "sa-random_swap" for problem: gr666: 0.26607s
[3/72] Solving TSP using "sa-random_swap" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "sa-random_swap" for problem: kroB100: 0.03463s
[4/72] Solving TSP using "sa-random_swap" for problem: d198 with 198 vertices...
Time taken to solve TSP using "sa-random_swap" for problem: d198: 0.06827s
[5/72] Solving TSP using "sa-random_swap" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "sa-random_swap" for problem: kroA150: 0.05170s
[6/72] Solving TSP using "sa-random_swap" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "sa-random_swap" for problem: tsp225: 0.07791s
[7/72] Solving TSP using "sa-random_swap" fo

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal,christofides_tour,christofides_cost,christofides_execution_time,christofides_deviation_from_optimal,sa-random_swap_tour,sa-random_swap_cost,sa-random_swap_execution_time,sa-random_swap_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,0.031263,23.981772,"[0, 48, 52, 119, 59, 50, 41, 43, 39, 46, 36, 9...",1.784470e+04,0.203121,192.057244,"[63, 75, 0, 82, 84, 89, 17, 39, 95, 20, 34, 26...",2.402026e+04,0.054080,293.130246
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,0.574678,24.665204,"[0, 665, 107, 108, 109, 110, 111, 449, 461, 46...",1.750465e+06,22.967080,494.672134,"[544, 357, 557, 495, 459, 504, 203, 434, 494, ...",3.434726e+06,0.266066,1066.853287
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,0.003091,31.678983,"[0, 89, 20, 16, 77, 12, 37, 19, 79, 29, 50, 47...",5.950318e+04,0.046793,168.746557,"[74, 87, 21, 18, 49, 53, 66, 25, 61, 38, 69, 6...",7.670735e+04,0.034630,246.449349
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,0.018013,17.997933,"[0, 119, 123, 124, 168, 61, 52, 136, 143, 144,...",7.218672e+04,0.844442,357.457031,"[149, 168, 154, 170, 128, 146, 152, 197, 112, ...",9.367511e+04,0.068269,493.631888
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,0.008833,26.714926,"[0, 70, 40, 65, 64, 123, 117, 127, 42, 96, 142...",1.203042e+05,0.340766,353.567357,"[103, 101, 50, 24, 80, 72, 129, 90, 106, 13, 4...",1.174192e+05,0.051700,342.690451


### Nearest Neighbor 2-Opt

In [12]:
solve_tsp(df_loaded, NearestNeighborTwoOptSimulatedAnnealingSolver(), SOLUTION_SIMULATED_ANNEALING_NEAREST_NEIGHBOR_2_OPT)

df_loaded.head()

[1/72] Solving TSP using "sa-nn_2_opt" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "sa-nn_2_opt" for problem: ch130: 0.09211s
[2/72] Solving TSP using "sa-nn_2_opt" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "sa-nn_2_opt" for problem: gr666: 0.31439s
[3/72] Solving TSP using "sa-nn_2_opt" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "sa-nn_2_opt" for problem: kroB100: 0.03597s
[4/72] Solving TSP using "sa-nn_2_opt" for problem: d198 with 198 vertices...
Time taken to solve TSP using "sa-nn_2_opt" for problem: d198: 0.07273s
[5/72] Solving TSP using "sa-nn_2_opt" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "sa-nn_2_opt" for problem: kroA150: 0.05453s
[6/72] Solving TSP using "sa-nn_2_opt" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "sa-nn_2_opt" for problem: tsp225: 0.08479s
[7/72] Solving TSP using "sa-nn_2_opt" for problem: fl417 with 417 vertices...
T

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,...,christofides_execution_time,christofides_deviation_from_optimal,sa-random_swap_tour,sa-random_swap_cost,sa-random_swap_execution_time,sa-random_swap_deviation_from_optimal,sa-nn_2_opt_tour,sa-nn_2_opt_cost,sa-nn_2_opt_execution_time,sa-nn_2_opt_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,...,0.203121,192.057244,"[63, 75, 0, 82, 84, 89, 17, 39, 95, 20, 34, 26...",2.402026e+04,0.054080,293.130246,"[95, 66, 12, 13, 9, 121, 54, 59, 50, 41, 43, 3...",8630.993515,0.092107,41.260123
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,...,22.967080,494.672134,"[544, 357, 557, 495, 459, 504, 203, 434, 494, ...",3.434726e+06,0.266066,1066.853287,"[651, 650, 649, 648, 643, 642, 641, 644, 645, ...",354781.000000,0.314387,20.527045
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,...,0.046793,168.746557,"[74, 87, 21, 18, 49, 53, 66, 25, 61, 38, 69, 6...",7.670735e+04,0.034630,246.449349,"[49, 42, 88, 86, 59, 73, 65, 3, 82, 5, 14, 32,...",26216.885561,0.035973,18.408769
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,...,0.844442,357.457031,"[149, 168, 154, 170, 128, 146, 152, 197, 112, ...",9.367511e+04,0.068269,493.631888,"[19, 18, 17, 22, 21, 23, 24, 25, 26, 27, 29, 2...",19012.697041,0.072725,20.486040
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,...,0.340766,353.567357,"[103, 101, 50, 24, 80, 72, 129, 90, 106, 13, 4...",1.174192e+05,0.051700,342.690451,"[2, 45, 28, 131, 111, 106, 120, 29, 100, 38, 9...",32827.547401,0.054533,23.765448


### Greedy Swap

In [13]:
solve_tsp(df_loaded, GreedySwapSimulatedAnnealingSolver(), SOLUTION_SIMULATED_ANNEALING_GREEDY_SWAP)

df_loaded.head()

[1/72] Solving TSP using "sa-greedy_swap" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "sa-greedy_swap" for problem: ch130: 0.05066s
[2/72] Solving TSP using "sa-greedy_swap" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "sa-greedy_swap" for problem: gr666: 0.31006s
[3/72] Solving TSP using "sa-greedy_swap" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "sa-greedy_swap" for problem: kroB100: 0.03475s
[4/72] Solving TSP using "sa-greedy_swap" for problem: d198 with 198 vertices...
Time taken to solve TSP using "sa-greedy_swap" for problem: d198: 0.07172s
[5/72] Solving TSP using "sa-greedy_swap" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "sa-greedy_swap" for problem: kroA150: 0.05338s
[6/72] Solving TSP using "sa-greedy_swap" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "sa-greedy_swap" for problem: tsp225: 0.09541s
[7/72] Solving TSP using "sa-greedy_swap" fo

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,...,sa-random_swap_execution_time,sa-random_swap_deviation_from_optimal,sa-nn_2_opt_tour,sa-nn_2_opt_cost,sa-nn_2_opt_execution_time,sa-nn_2_opt_deviation_from_optimal,sa-greedy_swap_tour,sa-greedy_swap_cost,sa-greedy_swap_execution_time,sa-greedy_swap_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,...,0.054080,293.130246,"[95, 66, 12, 13, 9, 121, 54, 59, 50, 41, 43, 3...",8630.993515,0.092107,41.260123,"[14, 23, 115, 94, 78, 37, 105, 57, 48, 52, 119...",7645.172213,0.050660,25.125568
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,...,0.266066,1066.853287,"[651, 650, 649, 648, 643, 642, 641, 644, 645, ...",354781.000000,0.314387,20.527045,"[491, 492, 518, 517, 516, 522, 520, 521, 524, ...",360342.000000,0.310062,22.416241
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,...,0.034630,246.449349,"[49, 42, 88, 86, 59, 73, 65, 3, 82, 5, 14, 32,...",26216.885561,0.035973,18.408769,"[81, 63, 13, 41, 1, 12, 77, 16, 44, 35, 95, 91...",27339.568667,0.034749,23.479376
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,...,0.068269,493.631888,"[19, 18, 17, 22, 21, 23, 24, 25, 26, 27, 29, 2...",19012.697041,0.072725,20.486040,"[102, 103, 104, 113, 112, 111, 105, 106, 110, ...",19189.171059,0.071721,21.604379
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,...,0.051700,342.690451,"[2, 45, 28, 131, 111, 106, 120, 29, 100, 38, 9...",32827.547401,0.054533,23.765448,"[115, 49, 43, 113, 143, 63, 39, 53, 1, 72, 67,...",34309.643590,0.053376,29.353203


### Logarithmic Cooling 2-Opt

In [14]:
solve_tsp(df_loaded, LogarithmicCoolingTwoOptSimulatedAnnealingSolver(), SOLUTION_SIMULATED_ANNEALING_LOGARITHMIC_COOLING_2_OPT)

df_loaded.head()

[1/72] Solving TSP using "sa-log_cool_2_opt" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "sa-log_cool_2_opt" for problem: ch130: 0.04829s
[2/72] Solving TSP using "sa-log_cool_2_opt" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "sa-log_cool_2_opt" for problem: gr666: 0.32199s
[3/72] Solving TSP using "sa-log_cool_2_opt" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "sa-log_cool_2_opt" for problem: kroB100: 0.03676s
[4/72] Solving TSP using "sa-log_cool_2_opt" for problem: d198 with 198 vertices...
Time taken to solve TSP using "sa-log_cool_2_opt" for problem: d198: 0.07095s
[5/72] Solving TSP using "sa-log_cool_2_opt" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "sa-log_cool_2_opt" for problem: kroA150: 0.05247s
[6/72] Solving TSP using "sa-log_cool_2_opt" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "sa-log_cool_2_opt" for problem: tsp225: 0.07884s
[7/72] S

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,...,sa-nn_2_opt_execution_time,sa-nn_2_opt_deviation_from_optimal,sa-greedy_swap_tour,sa-greedy_swap_cost,sa-greedy_swap_execution_time,sa-greedy_swap_deviation_from_optimal,sa-log_cool_2_opt_tour,sa-log_cool_2_opt_cost,sa-log_cool_2_opt_execution_time,sa-log_cool_2_opt_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,...,0.092107,41.260123,"[14, 23, 115, 94, 78, 37, 105, 57, 48, 52, 119...",7645.172213,0.050660,25.125568,"[97, 62, 24, 25, 108, 44, 85, 35, 91, 57, 54, ...",1.882679e+04,0.048289,208.130734
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,...,0.314387,20.527045,"[491, 492, 518, 517, 516, 522, 520, 521, 524, ...",360342.000000,0.310062,22.416241,"[469, 510, 590, 535, 525, 167, 297, 491, 463, ...",3.234994e+06,0.321990,998.999857
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,...,0.035973,18.408769,"[81, 63, 13, 41, 1, 12, 77, 16, 44, 35, 95, 91...",27339.568667,0.034749,23.479376,"[4, 38, 66, 61, 69, 72, 2, 20, 11, 84, 90, 27,...",5.477397e+04,0.036760,147.387060
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,...,0.072725,20.486040,"[102, 103, 104, 113, 112, 111, 105, 106, 110, ...",19189.171059,0.071721,21.604379,"[88, 65, 59, 71, 85, 98, 5, 54, 43, 55, 56, 11...",7.470373e+04,0.070955,373.407685
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,...,0.054533,23.765448,"[115, 49, 43, 113, 143, 63, 39, 53, 1, 72, 67,...",34309.643590,0.053376,29.353203,"[145, 4, 77, 45, 134, 88, 124, 63, 68, 53, 72,...",9.408644e+04,0.052470,254.721927


### Greedy 3-Opt

In [15]:
solve_tsp(df_loaded, GreedyThreeOptSimulatedAnnealingSolver(), SOLUTION_SIMULATED_ANNEALING_GREEDY_3_OPT)

df_loaded.head()

[1/72] Solving TSP using "sa-greedy_3_opt" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "sa-greedy_3_opt" for problem: ch130: 0.23851s
[2/72] Solving TSP using "sa-greedy_3_opt" for problem: gr666 with 666 vertices...
Time taken to solve TSP using "sa-greedy_3_opt" for problem: gr666: 1.07123s
[3/72] Solving TSP using "sa-greedy_3_opt" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "sa-greedy_3_opt" for problem: kroB100: 0.13457s
[4/72] Solving TSP using "sa-greedy_3_opt" for problem: d198 with 198 vertices...
Time taken to solve TSP using "sa-greedy_3_opt" for problem: d198: 0.27591s
[5/72] Solving TSP using "sa-greedy_3_opt" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "sa-greedy_3_opt" for problem: kroA150: 0.29227s
[6/72] Solving TSP using "sa-greedy_3_opt" for problem: tsp225 with 225 vertices...
Time taken to solve TSP using "sa-greedy_3_opt" for problem: tsp225: 0.31294s
[7/72] Solving TSP using "sa-gre

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,...,sa-greedy_swap_execution_time,sa-greedy_swap_deviation_from_optimal,sa-log_cool_2_opt_tour,sa-log_cool_2_opt_cost,sa-log_cool_2_opt_execution_time,sa-log_cool_2_opt_deviation_from_optimal,sa-greedy_3_opt_tour,sa-greedy_3_opt_cost,sa-greedy_3_opt_execution_time,sa-greedy_3_opt_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,...,0.050660,25.125568,"[97, 62, 24, 25, 108, 44, 85, 35, 91, 57, 54, ...",1.882679e+04,0.048289,208.130734,"[34, 117, 79, 45, 19, 92, 36, 21, 46, 39, 22, ...",8089.691895,0.238508,32.400849
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,...,0.310062,22.416241,"[469, 510, 590, 535, 525, 167, 297, 491, 463, ...",3.234994e+06,0.321990,998.999857,"[405, 407, 411, 413, 414, 415, 428, 429, 486, ...",359104.000000,1.071226,21.995665
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,...,0.034749,23.479376,"[4, 38, 66, 61, 69, 72, 2, 20, 11, 84, 90, 27,...",5.477397e+04,0.036760,147.387060,"[93, 56, 33, 6, 83, 57, 51, 53, 42, 88, 86, 59...",27814.449742,0.134567,25.624180
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,...,0.071721,21.604379,"[88, 65, 59, 71, 85, 98, 5, 54, 43, 55, 56, 11...",7.470373e+04,0.070955,373.407685,"[32, 33, 26, 25, 24, 23, 20, 21, 22, 28, 29, 2...",19228.313343,0.275910,21.852429
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,...,0.053376,29.353203,"[145, 4, 77, 45, 134, 88, 124, 63, 68, 53, 72,...",9.408644e+04,0.052470,254.721927,"[148, 54, 82, 33, 134, 139, 116, 6, 8, 56, 144...",33910.601529,0.292274,27.848747


## Lin-Kernighan Heuristic

In [16]:
solve_tsp(df_loaded, LinKernighanSolver(), SOLUTION_LIN_KERNIGHAN)

df_loaded.head()

[1/72] Solving TSP using "lk" for problem: ch130 with 130 vertices...
Time taken to solve TSP using "lk" for problem: ch130: 8.79959s
[2/72] Solving TSP using "lk" for problem: gr666 with 666 vertices...
Did not solve.
[3/72] Solving TSP using "lk" for problem: kroB100 with 100 vertices...
Time taken to solve TSP using "lk" for problem: kroB100: 1.99738s
[4/72] Solving TSP using "lk" for problem: d198 with 198 vertices...
Time taken to solve TSP using "lk" for problem: d198: 40.74034s
[5/72] Solving TSP using "lk" for problem: kroA150 with 150 vertices...
Time taken to solve TSP using "lk" for problem: kroA150: 14.12914s
[6/72] Solving TSP using "lk" for problem: tsp225 with 225 vertices...
Did not solve.
[7/72] Solving TSP using "lk" for problem: fl417 with 417 vertices...
Did not solve.
[8/72] Solving TSP using "lk" for problem: brg180 with 180 vertices...
Time taken to solve TSP using "lk" for problem: brg180: 2.21556s
[9/72] Solving TSP using "lk" for problem: brazil58 with 58 vert

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,...,sa-log_cool_2_opt_execution_time,sa-log_cool_2_opt_deviation_from_optimal,sa-greedy_3_opt_tour,sa-greedy_3_opt_cost,sa-greedy_3_opt_execution_time,sa-greedy_3_opt_deviation_from_optimal,lk_tour,lk_cost,lk_execution_time,lk_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,...,0.048289,208.130734,"[34, 117, 79, 45, 19, 92, 36, 21, 46, 39, 22, ...",8089.691895,0.238508,32.400849,"[60, 108, 60, 75, 10, 4, 44, 15, 127, 104, 111...",6395.491113,8.799592,4.672522
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,...,0.321990,998.999857,"[405, 407, 411, 413, 414, 415, 428, 429, 486, ...",359104.000000,1.071226,21.995665,None,NaN,NaN,NaN
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,...,0.036760,147.387060,"[93, 56, 33, 6, 83, 57, 51, 53, 42, 88, 86, 59...",27814.449742,0.134567,25.624180,"[15, 1, 41, 13, 63, 81, 32, 14, 5, 82, 3, 65, ...",21872.135370,1.997382,-1.214329
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,...,0.070955,373.407685,"[32, 33, 26, 25, 24, 23, 20, 21, 22, 28, 29, 2...",19228.313343,0.275910,21.852429,"[132, 131, 133, 137, 123, 168, 124, 125, 130, ...",15073.242060,40.740341,-4.478821
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,...,0.052470,254.721927,"[148, 54, 82, 33, 134, 139, 116, 6, 8, 56, 144...",33910.601529,0.292274,27.848747,"[140, 126, 58, 140, 73, 20, 71, 112, 9, 83, 35...",27463.368181,14.129137,3.541578


## Genetic Algorithms

### Tournament Selection

In [17]:
solve_tsp(df_loaded, TournamentSelectionGeneticAlgorithmSolver(), SOLUTION_GENETIC_TOURNAMENT_SELECTION)

df_loaded.head()

[1/72] Solving TSP using "ga-tournament" for problem: ch130 with 130 vertices...
Did not solve.
[2/72] Solving TSP using "ga-tournament" for problem: gr666 with 666 vertices...
Did not solve.
[3/72] Solving TSP using "ga-tournament" for problem: kroB100 with 100 vertices...
Did not solve.
[4/72] Solving TSP using "ga-tournament" for problem: d198 with 198 vertices...
Did not solve.
[5/72] Solving TSP using "ga-tournament" for problem: kroA150 with 150 vertices...
Did not solve.
[6/72] Solving TSP using "ga-tournament" for problem: tsp225 with 225 vertices...
Did not solve.
[7/72] Solving TSP using "ga-tournament" for problem: fl417 with 417 vertices...
Did not solve.
[8/72] Solving TSP using "ga-tournament" for problem: brg180 with 180 vertices...
Did not solve.
[9/72] Solving TSP using "ga-tournament" for problem: brazil58 with 58 vertices...
Did not solve.
[10/72] Solving TSP using "ga-tournament" for problem: lin105 with 105 vertices...
Did not solve.
[11/72] Solving TSP using "ga-t

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,...,sa-greedy_3_opt_execution_time,sa-greedy_3_opt_deviation_from_optimal,lk_tour,lk_cost,lk_execution_time,lk_deviation_from_optimal,ga-tournament_tour,ga-tournament_cost,ga-tournament_execution_time,ga-tournament_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,...,0.238508,32.400849,"[60, 108, 60, 75, 10, 4, 44, 15, 127, 104, 111...",6395.491113,8.799592,4.672522,None,NaN,NaN,NaN
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,...,1.071226,21.995665,None,NaN,NaN,NaN,None,NaN,NaN,NaN
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,...,0.134567,25.624180,"[15, 1, 41, 13, 63, 81, 32, 14, 5, 82, 3, 65, ...",21872.135370,1.997382,-1.214329,None,NaN,NaN,NaN
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,...,0.275910,21.852429,"[132, 131, 133, 137, 123, 168, 124, 125, 130, ...",15073.242060,40.740341,-4.478821,None,NaN,NaN,NaN
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,...,0.292274,27.848747,"[140, 126, 58, 140, 73, 20, 71, 112, 9, 83, 35...",27463.368181,14.129137,3.541578,None,NaN,NaN,NaN


### Adaptive Mutation

In [18]:
solve_tsp(df_loaded, AdaptiveMutationGeneticAlgorithmSolver(), SOLUTION_GENETIC_ADAPTIVE_MUTATION)

df_loaded.head()

[1/72] Solving TSP using "ga-adaptive" for problem: ch130 with 130 vertices...
Did not solve.
[2/72] Solving TSP using "ga-adaptive" for problem: gr666 with 666 vertices...
Did not solve.
[3/72] Solving TSP using "ga-adaptive" for problem: kroB100 with 100 vertices...
Did not solve.
[4/72] Solving TSP using "ga-adaptive" for problem: d198 with 198 vertices...
Did not solve.
[5/72] Solving TSP using "ga-adaptive" for problem: kroA150 with 150 vertices...
Did not solve.
[6/72] Solving TSP using "ga-adaptive" for problem: tsp225 with 225 vertices...
Did not solve.
[7/72] Solving TSP using "ga-adaptive" for problem: fl417 with 417 vertices...
Did not solve.
[8/72] Solving TSP using "ga-adaptive" for problem: brg180 with 180 vertices...
Did not solve.
[9/72] Solving TSP using "ga-adaptive" for problem: brazil58 with 58 vertices...
Did not solve.
[10/72] Solving TSP using "ga-adaptive" for problem: lin105 with 105 vertices...
Did not solve.
[11/72] Solving TSP using "ga-adaptive" for problem

,tsp_problem,number_of_vertices,graph,optimal_cost,dp_tour,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_tour,greedy_cost,...,lk_execution_time,lk_deviation_from_optimal,ga-tournament_tour,ga-tournament_cost,ga-tournament_execution_time,ga-tournament_deviation_from_optimal,ga-adaptive_tour,ga-adaptive_cost,ga-adaptive_execution_time,ga-adaptive_deviation_from_optimal
0,ch130,130,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6110.0,None,NaN,NaN,NaN,"[0, 40, 38, 70, 129, 49, 1, 117, 79, 45, 19, 3...",7575.286292,...,8.799592,4.672522,None,NaN,NaN,NaN,None,NaN,NaN,NaN
1,gr666,666,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",294358.0,None,NaN,NaN,NaN,"[0, 309, 308, 307, 301, 300, 299, 298, 296, 29...",366962.000000,...,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN
2,kroB100,100,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",22141.0,None,NaN,NaN,NaN,"[0, 94, 97, 11, 70, 26, 60, 34, 93, 56, 33, 6,...",29155.043704,...,1.997382,-1.214329,None,NaN,NaN,NaN,None,NaN,NaN,NaN
3,d198,198,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15780.0,None,NaN,NaN,NaN,"[0, 1, 6, 5, 2, 3, 4, 7, 8, 9, 10, 11, 12, 40,...",18620.073812,...,40.740341,-4.478821,None,NaN,NaN,NaN,None,NaN,NaN,NaN
4,kroA150,150,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26524.0,None,NaN,NaN,NaN,"[0, 129, 91, 7, 41, 121, 79, 30, 88, 132, 137,...",33609.866971,...,14.129137,3.541578,None,NaN,NaN,NaN,None,NaN,NaN,NaN


# Performance Analysis

## Summary Statistics

In [19]:
# Create the list of column names dynamically
metric_columns = []
for solution in ALL_SOLUTIONS:
	metric_columns.append(f'{solution}_cost')
	metric_columns.append(f'{solution}_execution_time')
	metric_columns.append(f'{solution}_deviation_from_optimal')

# Display summary statistics
df_loaded[metric_columns].describe()

,dp_cost,dp_execution_time,dp_deviation_from_optimal,greedy_cost,greedy_execution_time,greedy_deviation_from_optimal,christofides_cost,christofides_execution_time,christofides_deviation_from_optimal,sa-random_swap_cost,...,sa-greedy_3_opt_deviation_from_optimal,ga-tournament_cost,ga-tournament_execution_time,ga-tournament_deviation_from_optimal,ga-adaptive_cost,ga-adaptive_execution_time,ga-adaptive_deviation_from_optimal,lk_cost,lk_execution_time,lk_deviation_from_optimal
count,4.000000,4.000000,4.0,7.200000e+01,72.000000,72.000000,7.200000e+01,72.000000,72.000000,7.200000e+01,...,72.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,46.000000,46.000000,46.000000
mean,3076.500000,3.762833,0.0,3.876467e+05,0.118532,33.035781,2.784668e+06,4.285343,440.456731,5.753052e+06,...,24.808818,4009.071429,13.605595,0.068523,4012.785714,31.207137,0.122124,22022.784057,6.515823,-0.316826
std,2862.257093,2.671221,0.0,2.898035e+06,0.287192,61.879420,2.218057e+07,11.633368,1207.993402,4.636641e+07,...,29.362165,3679.072865,13.490782,0.210488,3686.091617,25.358554,0.312841,26758.496874,11.363148,7.606565
min,39.000000,0.491040,0.0,9.200000e+01,0.000041,3.496388,1.570000e+02,0.000553,20.061233,4.300000e+01,...,0.000000,39.000000,3.265305,0.000000,39.000000,10.367001,0.000000,30.000000,0.001107,-23.076923
25%,1573.500000,2.127552,0.0,8.039782e+03,0.002536,20.969953,1.497330e+04,0.029903,164.270783,1.623792e+04,...,17.988866,1357.250000,3.831847,0.000000,1357.250000,12.448813,0.000000,2461.549227,0.119077,-3.889147
50%,2704.000000,4.291139,0.0,2.690342e+04,0.008815,24.778315,8.036258e+04,0.156193,264.614001,1.025247e+05,...,23.812237,2396.000000,6.449158,0.000000,2396.000000,21.007656,0.000000,11264.000000,2.132424,1.259347
75%,4207.000000,5.926420,0.0,5.849190e+04,0.060736,28.145511,2.228753e+05,1.843113,440.318647,3.294747e+05,...,27.942790,6408.000000,20.328046,0.000000,6408.000000,38.466545,0.058895,27377.246608,7.709457,3.900777
max,6859.000000,5.978016,0.0,2.463096e+07,1.959548,533.846154,1.883642e+08,78.812198,10410.256410,3.936948e+08,...,255.384615,11461.000000,43.407679,0.780956,11470.000000,95.676332,1.166729,107463.446812,49.179728,16.135166


## Cost Analysis

In [20]:
# Prepare data for the plot
costs = ['optimal_cost', *(map(lambda solution: f'{solution}_cost', ALL_SOLUTIONS))]
legend = ['Optimal', *(map(lambda solution: solution.capitalize(), ALL_SOLUTIONS))]

# Create a new DataFrame to store the averaged results
averaged_data = df_loaded.groupby('number_of_vertices', as_index=False).mean(numeric_only=True)

# Create a Plotly figure
fig = go.Figure()

# Add traces for each solution's cost
for idx, cost in enumerate(costs):
	# Directly use the averaged cost values
	y_values = averaged_data[cost]

	# Create a scatter plot with lines connecting the points
	fig.add_trace(go.Scatter(
		x=averaged_data['number_of_vertices'],
		y=y_values,
		mode='lines+markers',
		name=legend[idx],
		line=dict(dash='dash' if 'Dp' in legend[idx] else 'solid'),  # Dashed line for DP
		marker=dict(size=8),
		connectgaps=True  # This option connects gaps in data points
	))

# Update layout
fig.update_layout(
	title='TSP Solution Costs vs. Optimal Solution Costs',
	xaxis_title='Number of Vertices',
	yaxis_title='Cost',
	legend_title='Solutions',
	hovermode='x unified',  # Show hover information for all traces at x
)

# Show the plot
fig.show()

## Time Complexity Analysis

In [21]:
def matplotlib_color_to_plotly(color):
	r, g, b, a = color
	return f'rgba({int(r * 255)}, {int(g * 255)}, {int(b * 255)}, {a})'

In [22]:
# Define model functions
def exponential_model(x, a, b):
	return a * np.exp(b * x)

def quadratic_model(x, a, b, c):
	return a * x**2 + b * x + c

def quadratic_log_model(x, a, b):
	return a * (x**2 * np.log(x)) + b

def cubic_model(x, a, b, c, d):
	return a * x**3 + b * x**2 + c * x + d

def linear_model(x, a, b):
	return a * x + b

In [34]:
def fit_and_plot_curve_fit(x_valid, y_valid, algorithm_name, color, fig):
	if algorithm_name in[
		SOLUTION_DP,
		SOLUTION_GENETIC_TOURNAMENT_SELECTION,
		SOLUTION_GENETIC_ADAPTIVE_MUTATION
	]:
		popt, _ = curve_fit(exponential_model, x_valid, y_valid, p0=[1, 0.1])
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = exponential_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Exp Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=color)  # Solid line
		))

	elif algorithm_name == SOLUTION_CHRISTOFIDES:  # Christofides (cubic)
		popt, _ = curve_fit(cubic_model, x_valid, y_valid)
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = cubic_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Cubic Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=color)  # Solid line
		))

	elif algorithm_name == SOLUTION_LIN_KERNIGHAN:  # Lin-Kernighan (N^2 log N)
		popt, _ = curve_fit(quadratic_log_model, x_valid, y_valid)
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = quadratic_log_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Quad Log Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=color)  # Solid line
		))

	elif algorithm_name in [
		SOLUTION_SIMULATED_ANNEALING_RANDOM_SWAP,
		SOLUTION_SIMULATED_ANNEALING_NEAREST_NEIGHBOR_2_OPT,
		SOLUTION_SIMULATED_ANNEALING_GREEDY_SWAP,
		SOLUTION_SIMULATED_ANNEALING_LOGARITHMIC_COOLING_2_OPT
	]:
		popt, _ = curve_fit(linear_model, x_valid, y_valid)
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = linear_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Linear Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=color)  # Solid line
		))

	else:  # Greedy or any other algorithm (quadratic)
		popt, _ = curve_fit(quadratic_model, x_valid, y_valid)
		x_fit = np.linspace(min(x_valid), max(x_valid), 100)
		y_fit = quadratic_model(x_fit, *popt)

		fig.add_trace(go.Scatter(
			x=x_fit,
			y=y_fit,
			mode='lines',
			name=f'{algorithm_name.capitalize()} (Quad Fit)',
			legendgroup=algorithm_name,  # Group data and curve under one legend
			line=dict(color=color)  # Solid line
		))

In [41]:
def plot_execution_times(df, solutions):
	execution_times = list(map(lambda solution: f'{solution}_execution_time', solutions))
	colors = px.colors.qualitative.Plotly  # Use Plotly's qualitative color palette

	fig = go.Figure()

	for idx, execution_time in enumerate(execution_times):
		algorithm_name = execution_time.replace('_execution_time', '')
		mask = df[execution_time].notna()
		x_valid = df.loc[mask, 'number_of_vertices']
		y_valid = df.loc[mask, execution_time]

		# Cycle through colors if there are more algorithms than colors in the palette
		plotly_color = colors[idx % len(colors)]

		# Scatter plot for the data points (hide legend for this trace)
		fig.add_trace(go.Scatter(
			x=x_valid,
			y=y_valid,
			mode='markers',
			name=f'{algorithm_name.capitalize()} (Data)',
			legendgroup=algorithm_name,
			showlegend=False,
			marker=dict(size=8, color=plotly_color)
		))

		# Fit and plot best-fit line based on the algorithm's complexity
		fit_and_plot_curve_fit(x_valid, y_valid, algorithm_name, plotly_color, fig)

	fig.update_layout(
		title='Execution Time vs. Number of Vertices for All Solutions',
		xaxis_title='Number of Vertices',
		yaxis_title='Execution Time (s)',
		legend_title='Solutions',
		hovermode='x unified'
	)

	fig.show()

In [42]:
plot_execution_times(df_loaded, ALL_SOLUTIONS)

## Deviation From Optimality

In [40]:
# Prepare data for the plot
deviations_from_optimal = list(map(lambda solution: f'{solution}_deviation_from_optimal', ALL_SOLUTIONS))
legend = list(map(lambda solution: solution.capitalize(), ALL_SOLUTIONS))

# Create a new DataFrame to store the averaged results
averaged_deviation_data = df_loaded.groupby('number_of_vertices', as_index=False).mean(numeric_only=True)

# Create a Plotly figure
fig = go.Figure()

# Add traces for each solution's deviation from optimal
for idx, deviation in enumerate(deviations_from_optimal):
	# Directly use the averaged deviation values
	y_values = averaged_deviation_data[deviation]

	# Create a scatter plot with lines connecting the points
	fig.add_trace(go.Scatter(
		x=averaged_deviation_data['number_of_vertices'],
		y=y_values,
		mode='lines+markers',
		name=legend[idx],
		line=dict(dash='dash' if 'Dp' in legend[idx] else 'solid'),  # Dashed line for DP
		marker=dict(size=8),
		connectgaps=True  # This option connects gaps in data points
	))

# Update layout
fig.update_layout(
	title='Deviation of TSP Solutions from Optimal',
	xaxis_title='Number of Vertices',
	yaxis_title='Deviation from Optimal (%)',
	legend_title='Solutions',
	hovermode='x unified',  # Show hover information for all traces at x
)

# Show the plot
fig.show()